# Tibber Python playground

## Load and fetch data from the Tibber API

In [ ]:
import json
import tibber
import pandas as pd
import sqlite3 as sql
from datetime import datetime, timedelta
import plotly.express as px
pd.options.plotting.backend = "plotly"

In [ ]:
with open("config.json", "r") as f:
    TIBBER = json.load(f)

In [ ]:
access_token = TIBBER["TIBBER_API_KEY"]
tibber_connection = tibber.Tibber(access_token)
await tibber_connection.update_info()
print(tibber_connection.name)

home = tibber_connection.get_homes()[0]
# await home.update_info()
# print(home.address1)

# await home.update_price_info()
# print(home.current_price_info)

In [ ]:
history = await home.get_historic_data(900)

In [ ]:
len(history)

In [ ]:
history[40]

In [ ]:
await tibber_connection.close_connection()

## Save to SQLite

In [ ]:
db = sql.connect(database='tibber.db')

In [ ]:
# df.to_sql(name='consumption', con=db, schema='home')

```json
{
    "from": "2022-09-08T16:00:00.000+02:00",
    "unitPrice": 3.9428125,
    "totalCost": 3.5238416666666668,
    "cost": 3.469675,
    "consumption": 0.88
}
```

In [ ]:
create_table = """
    CREATE TABLE IF NOT EXISTS consumption(
        start_time DATE PRIMARY KEY,
        unit_price REAL,
        total_cost REAL,
        cost REAL,
        consumption REAL
    );
"""

upsert_table = """
    INSERT INTO consumption(
        start_time
        , unit_price
        , total_cost
        , cost
        , consumption
    )
    VALUES(?, ?, ?, ?, ?)
    ON CONFLICT(start_time) DO UPDATE SET
        unit_price = excluded.unit_price
        , total_cost = excluded.total_cost
        , cost = excluded.cost
        , consumption = excluded.consumption;
"""

In [ ]:
db.execute(create_table)
db.executemany(upsert_table, [tuple(row.values()) for row in history])

In [ ]:
df = pd.read_sql_query(sql="select * from consumption order by start_time;", con=db)
# db.close()
df.head()

## Visualize

In [ ]:
df["start_time"] = pd.to_datetime(df["start_time"])
df.set_index("start_time", drop=True, inplace=True)

In [ ]:
df[["unit_price", "consumption"]].dtypes

In [ ]:
px.line(
    df, y=["unit_price","consumption"]
)

In [ ]:
current_time = datetime.now()
start_time = current_time - timedelta(days=1)
end_time = current_time + timedelta(days=1)
print(f"current_time: {current_time}\nstart_time: {start_time}\nend_time: {end_time}")

In [ ]:
# .between_time(start_time=start_time, end_time=end_time)